In [1]:
from flask import Flask
app = Flask(__name__)

@app.route('/')
def hello():
    return 'hello'
app.debug=True
app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000 (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: 1

c:\Users\goode\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\core\interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [4]:
from app import create_app

app = create_app()

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app 'app'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

In [ ]:
import torch
import clip 
